<a href="https://colab.research.google.com/github/Hopsakee/hello-world/blob/master/Crawl4AI_Scraping_with_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crawl4AI: Advanced Web Crawling and Data Extraction

- GitHub Repository: [https://github.com/unclecode/crawl4ai](https://github.com/unclecode/crawl4ai)

In [1]:
!sudo apt-get update && sudo apt-get install -y libwoff1 libopus0 libwebp6 libwebpdemux2 libenchant1c2a libgudev-1.0-0 libsecret-1-0 libhyphen0 libgdk-pixbuf2.0-0 libegl1 libnotify4 libxslt1.1 libevent-2.1-7 libgles2 libvpx6 libxcomposite1 libatk1.0-0 libatk-bridge2.0-0 libepoxy0 libgtk-3-0 libharfbuzz-icu0

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,105 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
G

In [2]:
# !pip install "crawl4ai @ git+https://github.com/unclecode/crawl4ai.git"
# !pip install "crawl4ai @ git+https://github.com/unclecode/crawl4ai.git@staging"
!pip install crawl4ai
!pip install nest-asyncio
!playwright install

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.1/94.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.0/616.0 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.1 MB/s eta 0:00:00
  Created wheel for html2text: filename=html2text-2024.2.26-py3-none-any.whl size=33111 sha256=8b6e3367d73dc8a15794db748421cf18921d75b0f3bd50125f2c3df4682baa38
  Stored in directory: /root/.cache/pip/wheels/f3/96/6d/a7eba8f80d31cbd188a2787b81514d82fc5ae6943c44777659
Successfully built html2text
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3

In [3]:
import asyncio
import nest_asyncio
from crawl4ai import AsyncWebCrawler
from crawl4ai.extraction_strategy import JsonCssExtractionStrategy, LLMExtractionStrategy
import json
import time
from pydantic import BaseModel, Field

nest_asyncio.apply()

In [ ]:
import os; os.environ['OPENAI_API_KEY'] = 'sk-proj-xxxx'

In [ ]:
import os
from google.colab import userdata
class OpenAIModelFee(BaseModel):
    player_name: str = Field(..., description="Name of the player")
    elo_score: str = Field(..., description="Elo Score of the player")
    games_number: str = Field(..., description="Number of games of the player")

async def extract_openai_fees():
    async with AsyncWebCrawler(verbose=True) as crawler:
        result = await crawler.arun(
            url='https://lichess.org/broadcast/fide-world-youth-chess-championships-2024--o18/round-1/0SPgwmAM#players',
            word_count_threshold=1,
            extraction_strategy=LLMExtractionStrategy(
                provider="openai/gpt-4o-mini-2024-07-18", api_token=os.getenv('OPENAI_API_KEY'),
                schema=OpenAIModelFee.schema(),
                extraction_type="schema",
                instruction="""From the crawled content, extract all mentioned player names along with their scores and games.
                Do not miss any models in the entire content. One extracted model JSON format should look like this:
                {"player_name": "Abdul M", "elo_score": "1400", "games_number": "0"}."""
            ),
            bypass_cache=True,
        )
        print((result.extracted_content))

# Uncomment the following line to run the OpenAI extraction example
await extract_openai_fees()

[LOG] 🌤️  Warming up the AsyncWebCrawler
[LOG] 🌞 AsyncWebCrawler is ready to crawl
[LOG] 🕸️ Crawling https://lichess.org/broadcast/fide-world-youth-chess-championships-2024--o18/round-1/0SPgwmAM#players using AsyncPlaywrightCrawlerStrategy...
[LOG] ✅ Crawled https://lichess.org/broadcast/fide-world-youth-chess-championships-2024--o18/round-1/0SPgwmAM#players successfully!
[LOG] 🚀 Crawling done for https://lichess.org/broadcast/fide-world-youth-chess-championships-2024--o18/round-1/0SPgwmAM#players, success: True, time taken: 2.82 seconds
[LOG] 🚀 Content extracted for https://lichess.org/broadcast/fide-world-youth-chess-championships-2024--o18/round-1/0SPgwmAM#players, success: True, time taken: 0.16 seconds
[LOG] 🔥 Extracting semantic blocks for https://lichess.org/broadcast/fide-world-youth-chess-championships-2024--o18/round-1/0SPgwmAM#players, Strategy: AsyncWebCrawler
[LOG] Call LLM for https://lichess.org/broadcast/fide-world-youth-chess-championships-2024--o18/round-1/0SPgwmAM#pl

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `PromptTokensDetails` but got `dict` with value `{'audio_tokens': None, 'cached_tokens': 0}` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
